# 7주차: 최종 미니 프로젝트 템플릿
## 주제: SQLite + 생성형 AI 기반 데이터 서비스 설계

이 노트북은 7주차 팀/개인 프로젝트를 위한 **템플릿**입니다.
아래 구조를 따라가며, 자신만의 작은 서비스를 설계·구현해 보세요.

예시 아이디어:
- 자연어로 질문하면 주문/매출 통계를 보여주는 대시보드 조교
- 고객 데이터를 설명해 주는 데이터 사전(data dictionary) 챗봇
- 상품 추천 결과를 보여 주고, LLM이 한국어로 설명해 주는 서비스


## 1. 프로젝트 개요 작성
- 팀원 이름
- 프로젝트 제목
- 해결하고 싶은 문제 / 시나리오
- 사용하려는 데이터(테이블) 소개


In [1]:
# 👉 여기에 프로젝트 개요를 작성하세요.
project_info = {
    "team_members": ["홍길동", "김영희"],
    "title": "예시: 온라인 쇼핑몰 주문 분석 조교",
    "goal": "자연어 질문으로 주문/매출 통계를 쉽게 조회할 수 있는 조교 만들기",
    "datasets": "customers, products, orders, order_items 테이블 사용"
}
project_info

{'team_members': ['홍길동', '김영희'],
 'title': '예시: 온라인 쇼핑몰 주문 분석 조교',
 'goal': '자연어 질문으로 주문/매출 통계를 쉽게 조회할 수 있는 조교 만들기',
 'datasets': 'customers, products, orders, order_items 테이블 사용'}

## 2. SQLite 데이터베이스 준비
### 옵션 A
- 수업에서 제공한 샘플 DB를 사용

### 옵션 B
- 팀이 직접 설계한 스키마로 새로운 DB를 만들고, 샘플 데이터를 삽입

아래 코드는 기본 샘플 DB를 만드는 예시입니다. 필요에 따라 수정하세요.

In [2]:
!pip install --quiet openai pandas

import os
import sqlite3
import pandas as pd
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = 'YOUR_KEY_IS_HERE'  # 실습 시 수정
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

conn = sqlite3.connect(':memory:')  # 필요한 경우 'project.db'로 파일 저장
cursor = conn.cursor()

# TODO: 아래 스키마/데이터를 팀 프로젝트에 맞게 수정하세요.
cursor.executescript(
    '''
    DROP TABLE IF EXISTS customers;
    DROP TABLE IF EXISTS products;
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS order_items;

    CREATE TABLE customers (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT UNIQUE,
        created_at TEXT
    );

    CREATE TABLE products (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        category TEXT,
        price REAL NOT NULL
    );

    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        order_date TEXT NOT NULL,
        status TEXT,
        FOREIGN KEY(customer_id) REFERENCES customers(id)
    );

    CREATE TABLE order_items (
        id INTEGER PRIMARY KEY,
        order_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        unit_price REAL NOT NULL,
        FOREIGN KEY(order_id) REFERENCES orders(id),
        FOREIGN KEY(product_id) REFERENCES products(id)
    );
    '''
)

# TODO: 샘플 데이터 삽입 (팀 프로젝트에 맞게 수정)
cursor.executescript(
    '''
    INSERT INTO customers (name, email, created_at) VALUES
        ('홍길동', 'hong@example.com', '2024-01-01'),
        ('김영희', 'kim@example.com', '2024-02-10');

    INSERT INTO products (name, category, price) VALUES
        ('노트북', '전자제품', 1500000),
        ('마우스', '전자제품', 25000),
        ('책상', '가구', 120000);

    INSERT INTO orders (customer_id, order_date, status) VALUES
        (1, '2024-03-01', '완료'),
        (2, '2024-03-05', '배송중');

    INSERT INTO order_items (order_id, product_id, quantity, unit_price) VALUES
        (1, 1, 1, 1500000),
        (1, 2, 2, 25000),
        (2, 3, 1, 120000);
    '''
)
conn.commit()
print('프로젝트용 DB 준비 완료')

프로젝트용 DB 준비 완료


## 3. 핵심 질의 정의하기
서비스가 지원해야 하는 **대표 질문 3~5개**를 정의해 보세요.
예시:
- "지난달 매출 상위 3개 상품은?"
- "가장 많이 주문한 고객은 누구인가?"
- "상품 카테고리별 평균 가격은?"

In [3]:
core_questions = [
    "예시: 지난달 매출 상위 3개 상품은?",
    "예시: 가장 많이 주문한 고객은 누구인가?",
    "예시: 상품 카테고리별 평균 가격은?",
]
core_questions

['예시: 지난달 매출 상위 3개 상품은?', '예시: 가장 많이 주문한 고객은 누구인가?', '예시: 상품 카테고리별 평균 가격은?']

## 4. LLM 연동 함수 템플릿
### 4-1. 자연어 → SQL 변환 함수
질문(한국어)을 입력하면 SQL과 한국어 설명을 JSON 형식으로 반환하도록 설계합니다.


In [4]:
def nl_to_sql(question: str) -> dict:
    system_prompt = """
당신은 SQLite 데이터베이스 조교입니다.
사용자의 질문을 분석하여 SQL을 생성하고 결과 의미를 한국어로 설명합니다.

⚠️ 반드시 다음 지침을 지키세요:
1) 응답은 무조건 JSON 형식만 출력해야 합니다.
2) JSON 밖의 설명, 코드블록, 글자, 공백도 출력하면 안 됩니다.
3) JSON 스키마:
{
  "sql": "SELECT ...",
  "explanation_ko": "..."
}
"""

    user_prompt = f"[질문]\n{question}"

    completion = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
        response_format={"type": "json_object"},  # ★ JSON 응답 강제
    )

    raw = completion.choices[0].message.content
    print("[LLM 원본 응답]\n", raw)

    # JSON 파싱
    import json
    try:
        return json.loads(raw)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        return {"sql": None, "explanation_ko": "JSON 파싱 실패"}


### 4-2. SQL 실행 + 결과를 LLM이 요약하도록 하기
1. `nl_to_sql`로 SQL을 생성
2. SQLite에서 실행해 `pandas.DataFrame`으로 결과 받기
3. 그 결과를 다시 LLM에 전달해, 자연어로 깔끔하게 설명하도록 할 수 있습니다.

In [6]:
# 👇 이 셀에서 직접 키를 적지 말고, Colab 메뉴의
#   [실행] → [환경변수 관리] 기능을 활용하는 것을 추천합니다.
# 예시) os.environ["OPENAI_API_KEY"] = "sk-..."  # <-- 실제 키로 교체

from google.colab import userdata

# Colab Secret에서 API Key 로딩
raw_key = userdata.get("OPENAI_API_KEY")
api_key = raw_key.strip() if raw_key else None

if api_key is None:
    raise ValueError(
        "❌ ERROR: Colab userdata에 OPENAI_API_KEY가 없습니다.\n"
        "Colab 왼쪽 메뉴 → 🔑 Secret → OPENAI_API_KEY 추가 후 런타임을 재시작하세요."
    )

# OpenAI 클라이언트 생성
client = OpenAI(api_key=api_key)
print("✅ OpenAI API Key Loaded (prefix):", api_key[:8], "***")

✅ OpenAI API Key Loaded (prefix): sk-proj- ***


In [7]:
def run_query_and_explain(question: str):
    parsed = nl_to_sql(question)
    sql = parsed.get("sql")
    base_explanation = parsed.get("explanation_ko")

    if not sql:
        print("SQL이 생성되지 않았습니다.")
        return

    print("\n[생성된 SQL]\n", sql)

    try:
        df = pd.read_sql_query(sql, conn)
    except Exception as e:
        print("SQL 실행 오류:", e)
        return

    print("\n[SQL 실행 결과]\n", df)

    # 결과를 LLM이 다시 설명하도록 요청할 수도 있음
    system_prompt = "당신은 데이터 분석 결과를 쉽게 설명해 주는 조교입니다."
    user_prompt = (
        "다음은 데이터베이스 질의 결과입니다. 이 결과가 의미하는 바를 요약해서 설명해 주세요.\n"
        f"[질문]\n{question}\n\n"
        f"[기본 설명]\n{base_explanation}\n\n"
        f"[결과 데이터]\n{df.to_markdown()}"
    )

    completion = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.3,
    )

    explanation = completion.choices[0].message.content
    print("\n[최종 한국어 설명]\n", explanation)

# 예시 실행 (실제 프로젝트에서 수정)
run_query_and_explain("홍길동 고객이 주문한 상품 개수는?")

[LLM 원본 응답]
 {
  "sql": "SELECT COUNT(*) FROM orders WHERE customer_name = '홍길동';",
  "explanation_ko": "홍길동 고객이 주문한 상품의 총 개수를 계산하는 쿼리입니다."
}

[생성된 SQL]
 SELECT COUNT(*) FROM orders WHERE customer_name = '홍길동';
SQL 실행 오류: Execution failed on sql 'SELECT COUNT(*) FROM orders WHERE customer_name = '홍길동';': no such column: customer_name


## 5. 사용자 인터페이스(간단 버전)
최소한의 텍스트 인터페이스를 만들어 봅니다.
- Colab에서 `input()`을 사용해도 되고,
- Streamlit, Gradio 등을 사용해 웹 UI를 만드는 것도 가능합니다(선택).

In [9]:
def interactive_loop():
    print("자연어 질문을 입력하면 DB를 조회하고 결과를 설명해 줍니다.")
    print("종료하려면 q 입력")
    while True:
        q = input("질문: ")
        if q.lower() in ["q", "quit", "exit"]:
            break
        run_query_and_explain(q)
        print("\n==============================\n")

# interactive_loop()  # 필요 시 주석 해제 후 실행

## 6. 평가 체크리스트
프로젝트를 마친 후, 다음 항목을 점검해 보세요.

1. **기능성**
   - [ ] 최소 3개 이상의 자연어 질문을 안정적으로 처리하는가?
   - [ ] 잘못된 질문/지원하지 않는 질문에 대해 안전하게 대응하는가?
2. **데이터베이스 활용**
   - [ ] JOIN, GROUP BY 등 기본 SQL 기능을 활용했는가?
   - [ ] 스키마 설계(또는 기존 스키마 이해)가 논리적인가?
3. **AI 활용**
   - [ ] LLM을 적절한 역할(예: SQL 생성, 결과 설명)에 사용했는가?
   - [ ] 환각/지어내기 가능성을 고려한 설계가 있는가?
4. **문서화**
   - [ ] 프로젝트 개요, 사용 데이터, 주요 질문, 한계점이 정리되어 있는가?

필요하다면 이 노트북을 복제하여 팀별로 수정·제출하면 됩니다.